# Langgraph example with LiteLLM and SAP LLMs

## [How Langgraph works](https://docs.langchain.com/oss/python/langgraph/quickstart#use-the-functional-api)

## Installation

In [2]:
%pip install -U langchain


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [1]:
%pip install -U langgraph

  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached zstandard-0.25.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.3 kB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
Using cached zstandard-0.25.0-cp312-cp312-macosx_11_0_arm64.whl (640 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
%pip install langchain-litellm

  Using cached langchain_core-0.3.79-py3-none-any.whl.metadata (3.2 kB)
  Using cached fastuuid-0.14.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (1.1 kB)
Using cached langchain_core-0.3.79-py3-none-any.whl (449 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 3.1 MB/s eta 0:00:00 0:00:01
Using cached fastuuid-0.14.0-cp312-cp312-macosx_11_0_arm64.whl (251 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.3
    Uninstalling langchain-core-1.0.3:
      Successfully uninstalled langchain-core-1.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 1.0.2 requires langchain-core>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain 1.0.3 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.

[notice] A new release of pip is avail

## Credentials for SAP Gen AI Hub
Get the service key from your SAP BTP tenant with AI subscription.

Add the following variables from the service key in a file called ".env" and put it in the same folder where you run the notebook:
```
"AICORE_AUTH_URL": "https://* * * .authentication.sap.hana.ondemand.com/oauth/token",
"AICORE_CLIENT_ID": " *** ",
"AICORE_CLIENT_SECRET": " *** ",
"AICORE_RESOURCE_GROUP": " *** ",
"AICORE_BASE_URL": "https://api.ai.***.cfapps.sap.hana.ondemand.com/
```

## Run the Langgraph with LiteLLM and SAP LLMs

In [ ]:
from langchain.tools import tool
from langchain_litellm import ChatLiteLLM
from langgraph.graph import add_messages
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    ToolCall,
)
from langchain_core.messages import BaseMessage
from langgraph.func import entrypoint, task
from dotenv import load_dotenv

Load your credentials as environment variables that Litellm can use automatically.

In [ ]:
load_dotenv()

Define the model with the SAP LLM.

In [ ]:
model = ChatLiteLLM(model="sap/gpt-4o", temperature=0)

Define the agent tool

In [ ]:
@tool
def get_weather(city: str):
    """
    Returns weather information for a given city.
    :param city:
    :return: weather information
    """
    city_normalized = city.lower().replace(" ", "")

    mock_weather_db = {
        "newyork": "The weather in New York is sunny with a temperature of 25°C.",
        "london": "It's cloudy in London with a temperature of 15°C.",
        "tokyo": "Tokyo is experiencing light rain and a temperature of 18°C.",
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return f"The weather in {city} is sunny with a temperature of 20°C."

Augment the LLM with tools

In [ ]:
tools = [get_weather]
tools_by_name = {tool.name: tool for tool in tools}
model_with_tools = model.bind_tools(tools)

Define model node

In [ ]:
@task
def call_llm(messages: list[BaseMessage]):
    """LLM decides whether to call a tool or not"""
    return model_with_tools.invoke(
        [
            SystemMessage(
                content="You are a helpful weather assistant. "
                "When the user asks you about a specific city, "
                "use the 'get_weather' tool to find the information about the weather. "
                "Answer with a TV weather report in two sentences, including a small joke."
            )
        ]
        + messages
    )

Define tool node

In [ ]:
@task
def call_tool(tool_call: ToolCall):
    """Performs the tool call"""
    tool = tools_by_name[tool_call["name"]]
    return tool.invoke(tool_call)

Define agent

In [ ]:
@entrypoint()
def agent(messages: list[BaseMessage]):
    model_response = call_llm(messages).result()

    while True:
        if not model_response.tool_calls:
            break

        # Execute tools
        tool_result_futures = [
            call_tool(tool_call) for tool_call in model_response.tool_calls
        ]
        tool_results = [fut.result() for fut in tool_result_futures]
        messages = add_messages(messages, [model_response, *tool_results])
        model_response = call_llm(messages).result()

    messages = add_messages(messages, model_response)
    return messages

User can select a city.

In [ ]:
city = 'London'

Run invoke

In [ ]:
input_message = [HumanMessage(content=f"What's the weather in {city}?")]
for chunk in agent.stream(input_message, stream_mode="updates"):
    print(chunk)
    print("\n")